In [1]:
import os
os.makedirs('output', exist_ok=True)

import sys
sys.path.append('../')

from utils import load_walksheds, overlay_wks
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import overlay
from shapely.geometry import Point, Polygon, LineString
import matplotlib.pyplot as plt
import fiona
import warnings
warnings.filterwarnings("ignore")

#### Read in Datasets and Files

* walskhedbuffer is the Metro station walksheds with overlapping boundaries
* all_boundaries All boundaries for census blocks in MD, DC and VA
* gdb_MD_BG, gdb_DC_BG, gdb_VA_BG are the block group geodataframes for MD, DC, VA
* ACS_income_MD, ACS_income_DC, ACS_income_VA are the ACS incomes for MD, DC, VA

In [2]:
wksp5, wksp75 = load_walksheds()
all_boundaries= gpd.read_file("../../Data/MD-DC-VA Boundaries.zip")


gdb_MD_BG = gpd.read_file("../../Data/tlgdb_2021_a_24_md.gdb", driver='FileGDB', layer='Block_Group')
gdb_DC_BG = gpd.read_file ("../../Data/tlgdb_2021_a_11_dc.gdb", driver='FileGDB', layer='Block_Group')
gdb_VA_BG = gpd.read_file ("../../Data/tlgdb_2021_a_51_va.gdb", driver='FileGDB', layer='Block_Group')

ACS_income_MD = gpd.read_file("../../Data/ACS_2020_5YR_BG_24_MARYLAND.gdb", driver='FileGDB', layer=15)
ACS_income_DC = gpd.read_file("../../Data/ACS_2020_5YR_BG_11_DISTRICT_OF_COLUMBIA.gdb", driver='FileGDB', layer=15)
ACS_income_VA = gpd.read_file("../../Data/ACS_2020_5YR_BG_51_VIRGINIA.gdb",driver='FileGDB', layer=15)


#### Preprocessing


* Remove "15000US" from values in the the GEOID column of the ACS dataframes
* Merge the ACS data from all three locations to the block groups based on the GEOID column
* Merging creates two geometry columns. Make the geometry_y column (from the block groups) the main geometry column and remove the geometry column from the ACS data

In [3]:
ACS_income_MD['GEOID'] = ACS_income_MD['GEOID'].str.replace('15000US', '')
ACS_income_DC['GEOID'] = ACS_income_DC['GEOID'].str.replace('15000US', '')
ACS_income_VA['GEOID'] = ACS_income_VA['GEOID'].str.replace('15000US', '')

In [4]:
ACS_Income_MD = pd.merge(ACS_income_MD, gdb_MD_BG, on='GEOID', how='outer')
ACS_Income_DC = pd.merge(ACS_income_DC, gdb_DC_BG, on='GEOID', how='outer')
ACS_Income_VA = pd.merge(ACS_income_VA, gdb_VA_BG, on='GEOID',how='outer')

In [5]:
ACS_Income_VA.rename(columns = {'geometry_y':'geometry'}, inplace = True)
ACS_Income_DC.rename(columns = {'geometry_y':'geometry'}, inplace = True)
ACS_Income_MD.rename(columns = {'geometry_y':'geometry'}, inplace = True)

ACS_Income_VA = ACS_Income_VA.drop(columns=['geometry_x'])
ACS_Income_DC = ACS_Income_DC.drop(columns=['geometry_x'])
ACS_Income_MD = ACS_Income_MD.drop(columns=['geometry_x'])

In [6]:
ACS_Income_MD.head(2)

,GEOID,B19001e1,B19001m1,B19001e2,B19001m2,B19001e3,B19001m3,B19001e4,B19001m4,B19001e5,...,B19313He1,B19313Hm1,B19313Ie1,B19313Im1,NAMELSAD,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,240010001001,370.0,104.0,3.0,7.0,0.0,13.0,19.0,22.0,54.0,...,20738500.0,7778983.0,NaN,NaN,Block Group 1,114609102.0,2207428.0,+39.6704707,-78.3878895,"MULTIPOLYGON (((-78.49625 39.60436, -78.49612 ..."
1,240010001002,793.0,140.0,63.0,43.0,71.0,61.0,10.0,16.0,0.0,...,49793100.0,12217270.0,NaN,NaN,Block Group 2,215159992.0,508136.0,+39.6796402,-78.5461225,"MULTIPOLYGON (((-78.66648 39.64913, -78.66643 ..."


* Merge the income variables from all three locations together
* Change the CRS of both the walksheds and the combined income variable
* Intersect the incomes with the walksheds
* Remove extra characters from the station names

In [7]:
## merge two ACE_income for MD, DC, VA ###
ACS_Income1 = pd.concat([ACS_Income_MD, ACS_Income_DC])
ACS_income = pd.concat([ACS_Income1, ACS_Income_VA])


In [8]:
ACS_income = ACS_income.to_crs('EPSG:4326')

In [9]:
intersect_income_stations_p5, intersect_income_stations_p75 = overlay_wks(ACS_income)

* Rename the B19013e1 column to median household income
* Group intersect_income_stations by the station names and summarize to get the median of the median household income
* Export the new variable

In [10]:
intersect_income_stations_p5.rename(columns = {'B19013e1':"Median_household_income_p5"}, inplace = True)
intersect_income_stations_p75.rename(columns = {'B19013e1':"Median_household_income_p75"}, inplace = True)

In [11]:
sum_income_stations_p5 = intersect_income_stations_p5.groupby(["Name_1"])["Median_household_income_p5"].median().reset_index()
sum_income_stations_p75 = intersect_income_stations_p75.groupby(["Name_1"])["Median_household_income_p75"].median().reset_index()
sum_income_stations_p5.head(2)

,Name_1,Median_household_income_p5
0,ADDISON ROAD-SEAT PLEASANT,77702.0
1,ANACOSTIA,36323.0


In [12]:
sum_income_stations_p5.to_excel("output/proportional_walkshed_household_income_updated_p5.xlsx", sheet_name=' Medianhouseholdincome', index=True)
sum_income_stations_p75.to_excel("output/proportional_walkshed_household_income_updated_p75.xlsx", sheet_name=' Medianhouseholdincome', index=True)
